In [11]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import copy
from IPython.display import Image
from qutip import *
import time
import math

In [12]:
dim = 2      # number of levels for each qubit
GHz = 1e8    #1e9 (0.1GHz)
ns = 1e-9
b = destroy(dim)                     # sigma minus operator

s0m = tensor(b, qeye(dim))           # sigma minus on 0
s0p = tensor(b.dag(), qeye(dim))     # sigma plus on 0
s1m = tensor(qeye(dim), b)           # sigma minus on 1
s1p = tensor(qeye(dim), b.dag())     # sigma plus on 1

omega = [2*np.pi*5.0*GHz, 2*np.pi*5.2*GHz]            # qubit transition frequencies [rad/s]
Delta = [2*np.pi*0*GHz,2*np.pi*0*GHz] #[-.33, -.33]   # anharmonicities [rad/s]
Omega = [2*np.pi*1*GHz, 2*np.pi*1.1*GHz]              # control-qubit drive strength [rad/s]
J = 2*np.pi*0.3*GHz                                     # passive interaction [rad/s]

c_max = 10  # upper bound for control amplitudes
c_min = 0   # lower bound for control amplitudes
T = 10*ns   # target time
L = 2     # for Gaussian, always set to 2

H0 = ( omega[0]*s0p*s0m + omega[1]*s1p*s1m ) + \
     (1/2)*( Delta[0]*s0p*s0p*s0m*s0m + Delta[1]*s1p*s1p*s1m*s1m ) + \
     J*(s0p*s1m+s0m*s1p)

Hc0m = Omega[0]*(s0m)
Hc1m = Omega[1]*(s1m)
Hc0p = Omega[0]*(s0p)
Hc1p = Omega[1]*(s1p)

'''
# Piecewise Constant Control on qubit 0
def C0(t, args):
    c = args['c']     # list of control parameters [c11,...,c1L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[0][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[0][L-1]*float(t==t_inv[L])  )

def C1(t, args):
    c = args['c']     # list of control parameters [c21,...,c2L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[1][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[1][L-1]*float(t==t_inv[L])  )

# Piecewise Constant Control on qubit 1
def C2(t, args):
    c = args['c']     # list of control parameters [c31,...,c3L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[2][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[2][L-1]*float(t==t_inv[L])  )

def C3(t, args):
    c = args['c']     # list of control parameters [c41,...,c4L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[3][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[3][L-1]*float(t==t_inv[L])  )
'''

# Gaussian Control on qubit 0
def C0(t, args):
    c = args['c']     # list of control parameters [c11, c12]
    T = args['T']     # target time
    return( c[0][0] * math.exp(-(t-(T/2))**2/((c[0][1])**2) * ( float(t>=0 and t<=T) ) ))
           
def C1(t, args):
    c = args['c']     # list of control parameters [c21, c22]
    T = args['T']     # target time
    return( c[1][0] * math.exp(-(t-(T/2))**2/((c[1][1])**2) * ( float(t>=0 and t<=T) ) ))

# Gaussian Control on qubit 1
def C2(t, args):
    c = args['c']     # list of control parameters [c11, c12]
    T = args['T']     # target time
    return( c[2][0] * math.exp(-(t-(T/2))**2/((c[2][1])**2) * ( float(t>=0 and t<=T) ) ))
           
def C3(t, args):
    c = args['c']     # list of control parameters [c21, c22]
    T = args['T']     # target time
    return( c[3][0] * math.exp(-(t-(T/2))**2/((c[3][1])**2) * ( float(t>=0 and t<=T) ) ))     
           

# Marker
def ep0(t, args):
    omega = args['omega']
    return np.exp(1j *omega[0]*t)

def em0(t, args):
    omega = args['omega']
    return np.exp(-1j *omega[0]*t)

def ep1(t, args):
    omega = args['omega']
    return np.exp(1j *omega[1]*t)

def em1(t, args):
    omega = args['omega']
    return np.exp(-1j *omega[1]*t)

# Control Functions C_i_j_k: i=0,1; j=m,p related to sm/sp; k=m,p related to (sm-sp) or (sm+sp)
def C0mm(t, args):
    return 1j*C0(t, args)*ep0(t, args)
def C0pm(t, args):
    return -1j*C0(t, args)*em0(t, args)

def C0mp(t, args):
    return C1(t, args)*ep0(t, args)
def C0pp(t, args):
    return C1(t, args)*em0(t, args)

def C1mm(t, args):
    return 1j*C2(t, args)*ep1(t, args)
def C1pm(t, args):
    return -1j*C2(t, args)*em1(t, args)

def C1mp(t, args):
    return C3(t, args)*ep1(t, args)
def C1pp(t, args):
    return C3(t, args)*em1(t, args)

# Total Hamiltonain
H = [H0, [Hc0m,C0mm], [Hc0p,C0pm], [Hc0m,C0mp], [Hc0p,C0pp], [Hc1m,C1mm], [Hc1p,C1pm], [Hc1m,C1mp], [Hc1p,C1pp]]   

In [13]:
GA = sigmax() # Target Unitary on qubit 0
GB = sigmax() # Target Unitary on qubit 1

In [14]:
def OBJ(c,T,L,omega,H,GA,GB):
    
    args = {
        'c': c,
        'T': T,
        'L': L,
        'omega': omega
    }
    
    U = propagator(H, T, args=args,options=Options(nsteps=1700))
    
    # SVD for partial traces
    sigma = np.sqrt((((tensor(GA,qeye(dim)).dag()*U).ptrace(1))*((tensor(GA,qeye(dim)).dag()*U).ptrace(1)).dag()).eigenstates()[0])
    lambd = np.sqrt((((tensor(qeye(dim),GB).dag()*U).ptrace(0))*((tensor(qeye(dim),GB).dag()*U).ptrace(0)).dag()).eigenstates()[0])
    
    # sum of fidelity
    return sum(sigma)/(2*dim**2)+sum(lambd)/(2*dim**2)

In [19]:
# GRadient Ascent Pulse Engineering (GRAPE)

def GRAPE(T,L,omega,H,GA,GB,c=None):
    eps = 0.01               # ascending rate
    dx = 0.01                # gradient increment
    threshold = 0.000001    # if cost changes less than threshold, then halt
    itera_max = 500          # max iteration number
    count = 0               # counting iterations
    diff_cost = np.inf      # initial cost difference
    d = T/L                 # time interval length
    record_cost = []
    factor = 0.5
    # Step1-1, Guess initial
    if c is None:
        c = [[float(np.random.uniform(c_min,c_max)) for _ in range(L)],    # for C1
             [float(np.random.uniform(c_min,c_max)) for _ in range(L)],    # for C2
             [float(np.random.uniform(c_min,c_max)) for _ in range(L)],    # for C3
             [float(np.random.uniform(c_min,c_max)) for _ in range(L)]]    # for C4
    # Step1-2, Compute initial cost
    cost = OBJ(c,T,L,omega,H,GA,GB)
    print(cost)
    record_cost.append(cost)
    # Start while loop
    while (count<=itera_max) and (diff_cost>threshold):
        #print(count)
        count += 1
        # Step 2, compute derivative numerically by Df(x0) = (f(x0+dx)-f(x0)) / dx
        D_J_c = [ [ 0 for l in range(L)] for m in range(4)]
        for m in range(4):
            for l in range(L):
                c_tmp1 = copy.deepcopy(c)
                c_tmp2 = copy.deepcopy(c)
                c_tmp1[m][l] += dx/2
                c_tmp2[m][l] -= dx/2
                D_J_c[m][l] += (OBJ(c_tmp1,T,L,omega,H,GA,GB) - OBJ(c_tmp2,T,L,omega,H,GA,GB)) / dx
        # Step 3, normalization for gradient
        norm2_D = 0
        for m in range(4):
            for l in range(L):
                norm2_D += D_J_c[m][l]**2
        norm_D = np.sqrt(norm2_D)
        D_J_c = [[eps*D_J_c[m][l]/norm_D for l in range(L)] for m in range(4)]
        # Step4, gradient ascend
        c_old = np.array(c)
        c = np.array(c) + np.array(D_J_c)
        # Step5, Compute cost
        new_cost = OBJ(c,T,L,omega,H,GA,GB)
        if new_cost > record_cost[-1]:
            print(new_cost)
            diff_cost = abs(new_cost - record_cost[-1])
            record_cost.append(new_cost)
        else:
            c = c_old
            dx *= factor
            eps *= factor
    return record_cost, c, count
    

In [ ]:
for i in range(2): 
    print('<'+str(i)+'>')
    fidelity,c, count = GRAPE(T,L,omega,H,GA,GB)
    print('------------')
    fidelity,c, count = GRAPE(T,L,omega,H,GA,GB,c)

<0>
0.3273154188159005
0.3507353739974258
0.3728652609072917
0.39361716553602516
0.41290967461861694
0.43066661844914184
0.44681754263536927
0.4612990322770562
0.47405411323292995
0.4850323370394046
0.4941904371410426
0.5014924079446713
0.5069097946040607
0.5104213746537642
0.5120148238455248
0.5120965711521216
0.512124770425646
0.5121345313886382
0.5121425543905211
0.5121499030988046
0.5121569545809157
0.5121638736781234
0.512170695072653
0.5121774421233402
0.512184076432058
0.5121907089299517
0.5121973340902658
0.5122037909232211
0.5122103505986002
0.5122167398315915
0.5122232339729177
0.5122296234550983
0.5122361121591311
0.5122424527035564
0.5122488776229148
0.512255228490509
0.5122616480307178
0.5122679984081138
0.512274418017982
0.5122807699746451
0.5122871917003163
0.5122935467217845
0.5122999718060621
0.5123063308139674
0.5123127600340593
0.5123191235990592
0.5123255574449022
0.5123319259219222
0.5123383647051982
0.5123447383169856
0.5123511822387644


In [14]:
#record_c = []
#record_f = {}
#n_trails = 1
#start_time = time.time()
#for i in range(n_trails): # apply GRAPE with random initial multiple times
    fidelity,c, count = GRAPE(T,L,omega,H,GA,GB)
    #record_f.update({i: fidelity})
    #record_c.append(c)
    #print(i) # check progress
    #plt.plot(fidelity)

#end_time = time.time()
#plt.legend([str(i) for i in range(n_trails)])
#plt.show()

0.6875802115989473
0.6933915298487843
0.6992154297317474
0.7046762104379847
0.7102423220037699
0.715650813467893
0.7209789518854683
0.7263878815217807
0.7316342454448681
0.7368130979161702
0.7419523093511051
0.746844852556096


In [15]:
fidelity,c, count = GRAPE(T,L,omega,H,GA,GB,c)

0.746844852556096
0.7519136045071416
0.7569816802838305
0.7616740458426028
0.7664087441303433
0.7710368832970415
0.7757851422754505
0.7803043177566802
0.784740251734318
0.7891760076983705
0.7934634180538458
0.7977718676023097
